In [1]:
# Ministry of Education - Kindergartens
# https://parents.education.gov.il/prhnet/gov-education/kindergarten/search-daycare

import dataflows as DF

url = 'https://parents.education.gov.il/prhnet/Api/MeonotController/GetExcel?0=2025&1=0&2=0&3=0&4=0&5=0&csrt=4274816473439949448'

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:136.0) Gecko/20100101 Firefox/136.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Content-Type': 'application/json',
    # 'Request-Id': '|imlaC.oxafW',
    # 'X-TS-AJAX-Request': 'true',
    # 'X-Security-CSRF-Token': '08a139caf1ab2800566c84b0c629a5432d57d519dc9eb1cde690348ead583e4305302c9484428e04c2acc71673da6d05',
    'Alt-Used': 'parents.education.gov.il',
    'Connection': 'keep-alive',
    'Referer': 'https://parents.education.gov.il/prhnet/gov-education/kindergarten/search-daycare',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Priority': 'u=0'
}

def join_address(address, city):
    if city not in address:
        return f'{address}, {city}'
    return address

DF.Flow(
    DF.load(url, format='html', http_headers=headers),
    DF.checkpoint('moe'),
    DF.add_field('moe_name', 'string', lambda r: r['שם וסמל מעון'].rsplit('-', 1)[0]),
    DF.add_field('moe_symbol', 'string', lambda r: r['שם וסמל מעון'].rsplit('-', 1)[1]),
    DF.add_field('moe_mol_symbol', 'string', lambda r: r['סמל זרוע העבודה']),
    DF.add_field('moe_owner', 'string', lambda r: r['בעלות']),
    DF.add_field('moe_sector', 'string', lambda r: r['מגזר']),
    DF.add_field('moe_license_Status', 'string', lambda r: r['סטטוס הרישוי']),
    DF.add_field('moe_city', 'string', lambda r: r['ישוב']),
    DF.add_field('moe_address', 'string', lambda r: join_address(r['כתובת'], r['moe_city'])),
    DF.add_field('moe_phone', 'string', lambda r: r['טלפון']),
    DF.add_field('moe_manager', 'string', lambda r: r['מנהל/ת המעון']),
    DF.select_fields(['[a-z_]+']),
    DF.printer()
).process()
    

using checkpoint data from .checkpoints/moe
GetExcel?0=2025&1=0&2=0&3=0&4=0&5=0&csrt=4274816473439949448:
#     moe_name                          moe_symbol  moe_mol_symbol    moe_owner                                       moe_sector    moe_city          moe_address                    moe_phone    moe_manager
      (string)                            (string)  (string)          (string)                                        (string)      (string)          (string)                       (string)     (string)
----  ------------------------------  ------------  ----------------  ----------------------------------------------  ------------  ----------------  -----------------------------  -----------  ------------------------
1     אורות(יסמין) -צפצפה                    53777  2618              הנהלת נעמ"ת                                     יהודי         אור עקיבא         הגפן 17, אור עקיבא             04-6260844   אלמקייס רז גרציה
2     אלמלאאכה אלפרדוס פרטי                  10274     

(<datapackage.package.Package at 0x129f6c4a0>, {})

In [1]:
import requests
import dataflows as DF

def get_data():
    url = 'https://daycareclasssearch.labor.gov.il/api/FramesSearch/GetFramesList'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:136.0) Gecko/20100101 Firefox/136.0',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Content-Type': 'application/json',
        'Connection': 'keep-alive',
        'Referer': 'https://daycareclasssearch.labor.gov.il/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Priority': 'u=0'
    }

    data = []
    for frame_type in range(1, 4):
        post_data = {"Year":2025,"FrameType":frame_type,"FrameId":""}
        response = requests.post(url, headers=headers, json=post_data, timeout=30)
        if response.status_code == 200:
            frames_list = response.json()['Frames']
            data.extend(frames_list)
        else:
            print("Request failed with status code:", response.status_code)
    return data

DF.Flow(
    get_data(),
    DF.checkpoint('labor'),
    DF.printer()
).process()

using checkpoint data from .checkpoints/labor
res_1:
#            Year      FrameId  FrameName              TownCode  TownName      Address             ManagerName            Phone          OrganizationId  OrganizationName             FrameTypeCode  FrameTypeDesc      TownDistrictCode  TownDistrictName      NumOfApprovedBabiesPlaces    NumOfAvailableBabiesPlaces    NumOfApprovedToddlersPlaces    NumOfAvailableToddlersPlaces    NumOfApprovedAdultsPlaces    NumOfAvailableAdultsPlaces    IsThereAdmissionsCommittee    DaycareType  DaycareTypeDesc
        (integer)    (integer)  (string)              (integer)  (string)      (string)            (string)               (string)            (integer)  (string)                         (integer)  (string)                  (integer)  (string)                              (integer)                     (integer)                      (integer)                       (integer)                    (integer)                     (integer)                  

(<datapackage.package.Package at 0x127dece60>, {})

In [1]:
# Ministry of Welfare - CKAN Datastore
# Sample access: https://data.gov.il/api/3/action/datastore_search?resource_id=de069ddf-bcbc-4754-bda0-84873a353f7b&limit=5

import requests
import dataflows as DF

def get_data():
    # Fetch data iteratively
    url = 'https://data.gov.il/api/3/action/datastore_search'
    resource_id = 'de069ddf-bcbc-4754-bda0-84873a353f7b'
    limit = 1000
    offset = 0
    data = []
    types = set()
    while True:
        params = {'resource_id': resource_id, 'limit': limit, 'offset': offset}
        response = requests.get(url, params=params, timeout=30)
        if response.status_code == 200:
            records = response.json()['result']['records']
            if len(records) == 0:
                break
            for record in records:
                if record.get('Type_Descr'):
                    if record['Type_Descr'] not in {
                        'מרכז הורים ילדים',
                        'מעון יום',
                        'מעון רב תכליתי',
                        'מועדונית',
                    }:
                        continue
                if record.get('Head_Department'):
                    if record['Head_Department'].strip() not in {
                        'אוטיסטים',
                        'נוער וצעירים',
                        'ילד ונוער',
                    }:
                        continue
                if record.get('From_Age') and record.get('From_Age') > 6:
                    continue
                data.append(record)
            offset += limit
        else:
            print("Request failed with status code:", response.status_code)
            break
    print(types)
    return data

DF.Flow(
    get_data(),
    DF.checkpoint('welfare'),
    DF.printer()
).process()


set()
using checkpoint data from .checkpoints/welfare
res_1:
#             _id    Misgeret_Id  Name                  Type_Descr        Head_Department    Second_Classific    Owner_Code_Descr    Organization                  ORGANIZATIONS_BUSINES_NUM    Registered_Business_Id  Authoritys        Authority_Id  Region_Descr      Actual_Capacity  Gender_Descr    From_Age    To_Age    Status_des    STARTD      Maneger_Name     Religion    Educate     City_Name      Adrees                 Telephone  Fax          Mailing_Box_Id    GisX         GisY         Target_Population_Descr
        (integer)      (integer)  (string)              (string)          (string)           (string)            (string)            (string)                                      (integer)                 (integer)  (string)             (integer)  (string)                (integer)  (string)        (any)       (any)     (string)      (string)    (string)         (string)    (string)    (string)       (string)          

(<datapackage.package.Package at 0x12fe0ce90>, {})